In [ ]:
! pip install lxml

In [28]:
import os
import xml.etree.ElementTree as ET

def get_assembly_name(csproj_content):
    assembly_name_start = csproj_content.find("<AssemblyName>") + len("<AssemblyName>")
    assembly_name_end = csproj_content.find("</AssemblyName>")
    return csproj_content[assembly_name_start:assembly_name_end].strip()

def find_referenced_assemblies(solution_path):
    assemblies = {}

    # Encontrar arquivos de projeto (.csproj) na solução
    for foldername, _, filenames in os.walk(solution_path):
        for filename in filenames:
            if filename.endswith('.csproj'):
                csproj_path = os.path.join(foldername, filename)

                # Obter o nome do projeto
                project_name = os.path.splitext(filename)[0]

                # Ler conteúdo do arquivo de projeto C#
                with open(csproj_path, 'r', encoding='utf-8') as file:
                    csproj_content = file.read()

                    # Obter o nome do assembly
                    assembly_name = get_assembly_name(csproj_content)
                    assemblies[project_name] = assembly_name

    return assemblies

def find_referenced_assemblies_in_references(solution_path, target_assembly_name):
    referenced_assemblies = {}

    # Encontrar arquivos de projeto (.csproj) na solução
    for foldername, _, filenames in os.walk(solution_path):
        for filename in filenames:
            if filename.endswith('.csproj'):
                csproj_path = os.path.join(foldername, filename)

                # Obter o nome do projeto
                project_name = os.path.splitext(filename)[0]

                # Ler conteúdo do arquivo de projeto C#
                with open(csproj_path, 'r', encoding='utf-8') as file:
                    csproj_content = file.read()

                    # Procurar referências ao assembly
                    root = ET.fromstring(csproj_content)
                    projectReferences = root.findall("./Project/ProjectReference")
                    references = [ref.find("Name").text.strip() for ref in projectReferences if ref is not None]
                    if target_assembly_name in references:
                        referenced_assemblies[project_name] = references

    return referenced_assemblies

def generate_assemblies_table(assemblies):
    print("| Projeto | Assembly Name |")
    print("|---------|---------------|")
    for project_name, assembly_name in assemblies.items():
        print(f"| {project_name} | {assembly_name} |")

In [29]:
# Substitua o caminho abaixo pelo caminho da sua solução C#
solution_path = r'C:\Sommus\SommusGestor-web\src'

assemblies = find_referenced_assemblies(solution_path)
target_assembly_name = list(assemblies.values())[0]  # Pegar o primeiro assembly como exemplo
referenced_assemblies = find_referenced_assemblies_in_references(solution_path, target_assembly_name)

generate_assemblies_table(referenced_assemblies)

KeyboardInterrupt: 